In [1]:
import logging
from selenium.webdriver.common.by import By
from selenium import webdriver
import time
import numpy as np
import pandas as pd
import threading
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [2]:
def loading_wait(find_func, arg):
    out = None
    while True:
        out = find_func(*arg)
        if out is not None:
            break
    return out


In [97]:
DRIVER_PATH = "/Users/vitvaraaravithya/Documents/airflow_final_project/driver/chromedriver"
ROOT_URL = "https://www.bblam.co.th/en/products/mutual-funds/historical-daily-navs?p_code=B-TREASURY&date_from=13&month_from=02&year_from=2016&date_to=31&month_to=12&year_to=2023"
options = webdriver.ChromeOptions()

browser_capabilities = {"browserName":"firefox","browserVersion":"112.0","platformName":"LINUX","se:noVncPort":7900,"se:vncEnabled":True}

driver = webdriver.Remote(
    command_executor="http://chrome:4444/wd/hub",
    desired_capabilities=browser_capabilities
)
driver.get(ROOT_URL)
div = loading_wait(driver.find_element, (By.CLASS_NAME, "btn-accept"))
div.click()

In [98]:
driver.set_window_size(1920, 1080)

In [99]:
selector = None
for i in loading_wait(driver.find_elements, (By.CLASS_NAME, "cs-placeholder")):
    if i.text == "B-TREASURY : Bualuang Treasury Fund":
        selector = i

selector.click()

In [100]:
selector = None
for i in loading_wait(driver.find_elements, (By.CLASS_NAME, "cs-placeholder")):
    if i.text == "B-TREASURY : Bualuang Treasury Fund":
        selector = i


In [101]:
fund_selector = None
for i in loading_wait(driver.find_elements, (By.TAG_NAME, "ul")):
    if "B-TREASURY" in i.text:
        fund_selector = i
        break

In [102]:
fund_goto = [i.text for i in loading_wait(fund_selector.find_elements, (By.TAG_NAME, 'li')) ]


In [90]:
driver.quit()

In [ ]:
from abc import ABC, abstractmethod
from typing import Callable
class SeleniumCrawler(ABC):

    def __init__(self, name: str,  root_url: str, driver_path: str) -> None:
        self.name = name
        self.root_url = root_url
        self.driver_path = driver_path
        self.driver = None

    @staticmethod
    def loading_wait(find_func: Callable, arg: tuple) -> object:
        out = None
        while True:
            out = find_func(*arg)
            if out is not None:
                break
        return out

    def get_page(self, page_url: str) -> None:
        self.driver.get(page_url)

    @abstractmethod
    def start(self) -> None:
        pass

    @abstractmethod
    def close(self) -> None:
        pass

    def status(self):
        status = 'inactive'
        if self.driver != None:
            status = "active"
        return status

    def __str__(self):
        status = self.status()
        return f"<SeleniumCrawler {self.name} status: {status}>"

In [ ]:
class FetchTableBualuang(SeleniumCrawler):
    def __init__(self, driver_path: str, root_url: str, funds_name: str):
        self.funds_name = funds_name

        self.driver = None
        self.table = None

    def start(self):

        # Goto Root Url
        self.get_page(self.root_url)

        # Close Popup info
        div = self.loading_wait(self.driver.find_element, (By.CLASS_NAME, "btn-accept"))
        div.click()

        # Click fund in Nav bar
        self.click_fund_selector()
        self.select_fund()

        # Crawing table
        self.crawing_table()

    def click_fund_selector(self):
        selector = None
        for i in self.loading_wait(self.driver.find_elements, (By.CLASS_NAME, "cs-placeholder")):
            if i.text == "B-TREASURY : Bualuang Treasury Fund":
                selector = i
                break
        selector.click()

    def select_fund(self):
        fund_selector = None
        for i in self.loading_wait(self.driver.find_elements,(By.TAG_NAME, "ul")):
            if "B-TREASURY" in i.text:
                fund_selector = i
                break
        self.click_fund(fund_selector)
        self.click_submit_fund()

    def click_fund(self, fund_selector):
        fund_target = None
        for i in self.loading_wait(fund_selector.find_elements,(By.TAG_NAME, 'li')):
            if self.funds_name in i.text:
                fund_target = i
                break
        fund_target.click()

    def click_submit_fund(self):
        self.loading_wait(self.driver.find_elements,(By.CLASS_NAME, 'submit'))[1].click()

    def crawing_table(self):
        script = """
        var rows = document.querySelectorAll("table tr");
        var filteredRows = [];
        for (var i = 1; i < rows.length; i++) {
            var cells = rows[i].querySelectorAll("td");
            if (cells.length === 6 && cells[1].textContent.trim() !== "") {
                var row = [];
                for (var j = 0; j < cells.length; j++) {
                    row.push(cells[j].textContent.trim());
                }
                filteredRows.push(row);
            }
        }
        return filteredRows;
        """
        table_rows = self.driver.execute_script(script)

        # To dataframe
        schemas = ['date','NAV', 'NavperUnit', 'sellingPrice', 'redemptionPrice', 'change']
        df = pd.DataFrame(table_rows, columns=schemas)

        # To float
        df.replace('N/A', None, inplace=True)
        numerical = ['NAV', 'NavperUnit', 'sellingPrice', 'redemptionPrice', 'change']
        for i in numerical:
            df[i] = df[i].str.replace(',', "")
            df[i] = df[i].astype(float)
        df['fund_name'] = self.funds_name
        self.table = df.copy()

    def close(self):
        self.driver.close()

In [ ]:
FetchTable = FetchTableBualuang(driver_path=DRIVER_PATH, root_url=ROOT_URL, funds_name=fund_goto[1])

In [ ]:
FetchTable.start()

In [ ]:
FetchTable.close()

In [ ]:

FetchTable.table

$L(y) = a\frac{d^2y}{dx^2} + \frac{dy}{dx}+cy$